In [599]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.pyplot
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


#### Manipulation of dataframes with dog areas information

The scope of this notebook is to manipulate data found on https://dati.comune.milano.it/ with geograpphical coordinates, number and dimension of dog areas in Milano.
The dataset found have been manipulate and merged to obtain a final dataset containing aggregate information about dog areas in each NIL.

In [600]:
df_dog_areas=pd.read_csv('patrimonio_verde_2019_layer_4_area-cani_municipi_nil_wm_4326_final.csv',sep=';')

In [601]:
print(df_dog_areas.shape)
df_dog_areas.head()

(383, 7)


,area_mq,MUNICIPIO,ID_NIL,NIL,LONG_X_4326_CENTROID,LAT_Y_4326_CENTROID,Location
0,2046.45650,6,50,PORTA GENOVA,9.166443,45.458830,"(45.45882984119884, 9.166442945360316)"
1,1091.67340,6,50,PORTA GENOVA,9.166286,45.456674,"(45.45667406631106, 9.166285743040401)"
2,1565.74150,6,50,PORTA GENOVA,9.167216,45.456918,"(45.45691805045791, 9.16721624969385)"
3,626.51320,4,25,CORSICA,9.229143,45.467944,"(45.46794386816213, 9.22914298276349)"
4,296.62265,6,53,LORENTEGGIO,9.125099,45.448788,"(45.44878788764096, 9.125098822776701)"


In [602]:
df_dog_areas.rename(columns={df_dog_areas.columns[0]: 'dog_area_mq',
                      df_dog_areas.columns[1]: 'MUN',
                      df_dog_areas.columns[3]: 'NIL',
                      df_dog_areas.columns[4]: 'dog_area_long',
                      df_dog_areas.columns[5]: 'dog_area_lat'
                      },inplace=True)

df_dog_areas.drop(columns=df_dog_areas.columns[-1],inplace=True)

print(df_dog_areas.shape)
df_dog_areas.head()

(383, 6)


,dog_area_mq,MUN,ID_NIL,NIL,dog_area_long,dog_area_lat
0,2046.45650,6,50,PORTA GENOVA,9.166443,45.458830
1,1091.67340,6,50,PORTA GENOVA,9.166286,45.456674
2,1565.74150,6,50,PORTA GENOVA,9.167216,45.456918
3,626.51320,4,25,CORSICA,9.229143,45.467944
4,296.62265,6,53,LORENTEGGIO,9.125099,45.448788


In [603]:
print(type(df_dog_areas['MUN'][0]))
print(type(df_dog_areas['ID_NIL'][0]))
print(type(df_dog_areas['dog_area_mq'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>


In [604]:
df_dog_areas.sort_values(by=['ID_NIL','dog_area_mq'],inplace=True)
df_dog_areas.reset_index(inplace=True,drop=True)

print(df_dog_areas.shape)
df_dog_areas.head()

(383, 6)


,dog_area_mq,MUN,ID_NIL,NIL,dog_area_long,dog_area_lat
0,1567.106991,1,1,DUOMO,9.183142,45.458010
1,106.817333,1,2,BRERA,9.190211,45.480290
2,1256.636866,1,2,BRERA,9.190329,45.477792
3,4657.173800,1,3,GIARDINI P.TA VENEZIA,9.198839,45.475930
4,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107


In [605]:
print(df_dog_areas['ID_NIL'].unique())
print(df_dog_areas['NIL'].unique())

print(len(df_dog_areas['ID_NIL'].unique()))
print(len(df_dog_areas['NIL'].unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 35 36 37 38 40 41 42 43 44 45 46 48 49 50 51
 52 53 55 56 57 58 59 60 61 62 65 66 67 68 69 70 71 72 76 77 78 79 80 81
 82 83 84 87]
['DUOMO' 'BRERA' 'GIARDINI P.TA VENEZIA' 'GUASTALLA'
 'PORTA VIGENTINA - PORTA LODOVICA' 'PORTA TICINESE - CONCA DEL NAVIGLIO'
 'MAGENTA - S. VITTORE' 'PARCO SEMPIONE' 'PORTA GARIBALDI - PORTA NUOVA'
 'STAZIONE CENTRALE - PONTE SEVESO' 'ISOLA' 'MACIACHINI - MAGGIOLINA'
 'GRECO - SEGNANO'
 "NIGUARDA - CA' GRANDA - PRATO CENTENARO - Q.RE FULVIO TESTI" 'BICOCCA'
 'GORLA - PRECOTTO' 'ADRIANO' 'CIMIANO - ROTTOLE - Q.RE FELTRE'
 'PADOVA - TURRO - CRESCENZAGO' 'LORETO - CASORETTO - NOLO'
 'BUENOS AIRES - PORTA VENEZIA - PORTA MONFORTE' "CITTA' STUDI"
 'LAMBRATE - ORTICA' 'PARCO FORLANINI - CAVRIANO' 'CORSICA' 'XXII MARZO'
 'PTA ROMANA' 'UMBRIA - MOLISE - CALVAIRATE' 'ORTOMERCATO'
 'TALIEDO - MORSENCHIO - Q.RE FORLANINI' "MONLUE' - PONTE LAMBRO"
 'TRIULZO SUPERIORE

In [606]:
address = 'Milano, Italy'

geolocator = Nominatim(user_agent="MI_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milano, Italy are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milano, Italy are 45.4668, 9.1905.


In [607]:
# create map of Milano using latitude and longitude values
map_milano = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(1,9)
ys = [i + x + (i*x)**2 for i in range(9)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, municipio, nil,id_nil in zip(df_dog_areas['dog_area_lat'], df_dog_areas['dog_area_long'], df_dog_areas['MUN'], df_dog_areas['NIL'],df_dog_areas['ID_NIL']):
    label = 'NIL id: {}, NIL name:{}, Municipio {}'.format(id_nil,nil, municipio)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[municipio-1],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milano)  
    
map_milano

In [608]:
df_dog_areas.to_csv('dog_areas_data.csv',index=False)

In [609]:
print(df_dog_areas.shape)
df_dog_areas.head()

(383, 6)


,dog_area_mq,MUN,ID_NIL,NIL,dog_area_long,dog_area_lat
0,1567.106991,1,1,DUOMO,9.183142,45.458010
1,106.817333,1,2,BRERA,9.190211,45.480290
2,1256.636866,1,2,BRERA,9.190329,45.477792
3,4657.173800,1,3,GIARDINI P.TA VENEZIA,9.198839,45.475930
4,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107


In [610]:
print(df_dog_areas.groupby(['ID_NIL'])['dog_area_mq'].sum())
dog_area_sum=df_dog_areas.groupby(['ID_NIL'])['dog_area_mq'].sum().tolist()
dog_area_sum

ID_NIL
1      1567.106991
2      1363.454199
3     12599.599350
4       996.685367
5     10534.321173
6      4387.577287
7      1315.876399
8     25919.363206
9      1550.306013
10      555.935600
11     5255.673084
12     6909.022225
13     1776.735049
14    75574.861798
15     3076.189126
16    12037.978956
17     9905.446242
18    10220.629229
19    12595.240265
20     3699.560708
21     6021.801435
22     6193.978415
23     4667.631282
24     4119.003651
25     2644.767360
26    14849.862386
27     2533.719387
28     2524.816428
29     7884.925857
30     9712.421216
31      915.086758
32     1408.989531
33     1851.802532
35     8656.323386
36     6312.095128
37     3292.749414
38     9161.524537
40      516.820700
41     7385.327332
42    10209.429003
43     3891.290554
44     3372.209312
45     2826.820145
46     9876.939860
48     8254.196883
49    11696.764847
50     6709.779077
51     4596.217325
52     5964.281352
53    19464.985687
55    13898.163448
56    11967.287348
57   

[1567.1069908496802,
 1363.454199132053,
 12599.59934999681,
 996.6853669292933,
 10534.321172875218,
 4387.57728661893,
 1315.876398747395,
 25919.36320577631,
 1550.306012637615,
 555.935599998234,
 5255.673083648593,
 6909.022225361727,
 1776.7350485687862,
 75574.86179828839,
 3076.1891259225013,
 12037.978956475261,
 9905.446242041877,
 10220.62922947856,
 12595.240265298988,
 3699.560707517605,
 6021.801435208282,
 6193.978415259639,
 4667.631282243162,
 4119.0036508242,
 2644.7673596031977,
 14849.862385594508,
 2533.719387142179,
 2524.816428420762,
 7884.925857350161,
 9712.421216148252,
 915.086758407727,
 1408.989531300326,
 1851.8025320725012,
 8656.323386203037,
 6312.095128431127,
 3292.7494140460803,
 9161.52453739969,
 516.820699984242,
 7385.327332204521,
 10209.429002875771,
 3891.29055430012,
 3372.209312052806,
 2826.820144839989,
 9876.939859832819,
 8254.196882983473,
 11696.764846811706,
 6709.779076691472,
 4596.217325441233,
 5964.281351810578,
 19464.985686702

In [611]:
print(df_dog_areas.groupby('ID_NIL').size())
num_dog_area=df_dog_areas.groupby('ID_NIL').size().tolist()
num_dog_area

ID_NIL
1      1
2      2
3      2
4      3
5      6
6      4
7      5
8      4
9      2
10     1
11     6
12     5
13     3
14    19
15     4
16     9
17     5
18     6
19     8
20     4
21    11
22     7
23     4
24     2
25     4
26     7
27     3
28     4
29     3
30    10
31     2
32     3
33     3
35     9
36     5
37     2
38     8
40     1
41     8
42    11
43     3
44     5
45     2
46     9
48     8
49    11
50     5
51     5
52     8
53    12
55     9
56    10
57     3
58     1
59     1
60     4
61     5
62     2
65     9
66     2
67     2
68     3
69     2
70     4
71     8
72     4
76     9
77     3
78     2
79     5
80     4
81     3
82     2
83     4
84     7
87     1
dtype: int64


[1,
 2,
 2,
 3,
 6,
 4,
 5,
 4,
 2,
 1,
 6,
 5,
 3,
 19,
 4,
 9,
 5,
 6,
 8,
 4,
 11,
 7,
 4,
 2,
 4,
 7,
 3,
 4,
 3,
 10,
 2,
 3,
 3,
 9,
 5,
 2,
 8,
 1,
 8,
 11,
 3,
 5,
 2,
 9,
 8,
 11,
 5,
 5,
 8,
 12,
 9,
 10,
 3,
 1,
 1,
 4,
 5,
 2,
 9,
 2,
 2,
 3,
 2,
 4,
 8,
 4,
 9,
 3,
 2,
 5,
 4,
 3,
 2,
 4,
 7,
 1]

In [612]:
df_aggr_dog=df_dog_areas.drop_duplicates(subset=['NIL'],keep='last').reset_index(drop=True)
print(df_aggr_dog.shape)
df_aggr_dog.head()

(76, 6)


,dog_area_mq,MUN,ID_NIL,NIL,dog_area_long,dog_area_lat
0,1567.106991,1,1,DUOMO,9.183142,45.458010
1,1256.636866,1,2,BRERA,9.190329,45.477792
2,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107
3,549.244339,1,4,GUASTALLA,9.202896,45.454228
4,4703.437904,5,5,PORTA VIGENTINA - PORTA LODOVICA,9.193120,45.447192


In [613]:
df_aggr_dog.rename(columns={df_aggr_dog.columns[0]: 'bigger_dog_area_mq',
                     df_aggr_dog.columns[-2]: 'bigger_dog_area_long',
                     df_aggr_dog.columns[-1]: 'bigger_dog_area_lat'},inplace=True)

print(df_aggr_dog.shape)
df_aggr_dog.head()

(76, 6)


,bigger_dog_area_mq,MUN,ID_NIL,NIL,bigger_dog_area_long,bigger_dog_area_lat
0,1567.106991,1,1,DUOMO,9.183142,45.458010
1,1256.636866,1,2,BRERA,9.190329,45.477792
2,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107
3,549.244339,1,4,GUASTALLA,9.202896,45.454228
4,4703.437904,5,5,PORTA VIGENTINA - PORTA LODOVICA,9.193120,45.447192


In [614]:
print(df_dog_areas.groupby(['ID_NIL'])['dog_area_mq'].sum()) #stored in dog_area_sum
print(df_dog_areas.groupby('ID_NIL').size()) #stored in num_dog_area
df_aggr_dog[['ID_NIL']]

ID_NIL
1      1567.106991
2      1363.454199
3     12599.599350
4       996.685367
5     10534.321173
6      4387.577287
7      1315.876399
8     25919.363206
9      1550.306013
10      555.935600
11     5255.673084
12     6909.022225
13     1776.735049
14    75574.861798
15     3076.189126
16    12037.978956
17     9905.446242
18    10220.629229
19    12595.240265
20     3699.560708
21     6021.801435
22     6193.978415
23     4667.631282
24     4119.003651
25     2644.767360
26    14849.862386
27     2533.719387
28     2524.816428
29     7884.925857
30     9712.421216
31      915.086758
32     1408.989531
33     1851.802532
35     8656.323386
36     6312.095128
37     3292.749414
38     9161.524537
40      516.820700
41     7385.327332
42    10209.429003
43     3891.290554
44     3372.209312
45     2826.820145
46     9876.939860
48     8254.196883
49    11696.764847
50     6709.779077
51     4596.217325
52     5964.281352
53    19464.985687
55    13898.163448
56    11967.287348
57   

,ID_NIL
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [615]:
df_aggr_dog['num_dog_area']=num_dog_area
df_aggr_dog['sum_dog_area_mq']=dog_area_sum

print(df_aggr_dog.shape)
df_aggr_dog.head()

(76, 8)


,bigger_dog_area_mq,MUN,ID_NIL,NIL,bigger_dog_area_long,bigger_dog_area_lat,num_dog_area,sum_dog_area_mq
0,1567.106991,1,1,DUOMO,9.183142,45.458010,1,1567.106991
1,1256.636866,1,2,BRERA,9.190329,45.477792,2,1363.454199
2,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107,2,12599.599350
3,549.244339,1,4,GUASTALLA,9.202896,45.454228,3,996.685367
4,4703.437904,5,5,PORTA VIGENTINA - PORTA LODOVICA,9.193120,45.447192,6,10534.321173


In [616]:
df_aggr_dog.to_csv('aggr_dog_areas_data.csv',index=False)

In [617]:
print(df_aggr_dog.shape)
df_aggr_dog.head()

(76, 8)


,bigger_dog_area_mq,MUN,ID_NIL,NIL,bigger_dog_area_long,bigger_dog_area_lat,num_dog_area,sum_dog_area_mq
0,1567.106991,1,1,DUOMO,9.183142,45.458010,1,1567.106991
1,1256.636866,1,2,BRERA,9.190329,45.477792,2,1363.454199
2,7942.425550,1,3,GIARDINI P.TA VENEZIA,9.198413,45.474107,2,12599.599350
3,549.244339,1,4,GUASTALLA,9.202896,45.454228,3,996.685367
4,4703.437904,5,5,PORTA VIGENTINA - PORTA LODOVICA,9.193120,45.447192,6,10534.321173
